In [1]:
from tools import *
import random

In [2]:
GD_SVC = GoogleDrive()
SONG_FORMATS = ['flac', 'm4a', 'mp3', 'wav']
MUSIC_FOLDER = '1XS5yDT8Ym3ME0SUywRuMFm5AGxaSIzaN'

In [3]:
def choose_song(parent_folder_id):

    files = GD_SVC.list_files_on_folder(parent_folder_id)
    songs = [f for f in files if f['name'].split('.')[-1] in SONG_FORMATS]
    song_names = sorted([f['name'] for f in files if f['name'].split('.')[-1] in SONG_FORMATS])
    
    while True:
        choice_str = """
Choose a song from the below list:
0) Playlist with all songs in order
1) Playlist with all songs shuffled
2) All album glued as one continuous song
"""
        for i, n in enumerate(song_names):
            choice_str += "{0}) {1}\n".format(i+3, n)
        print(choice_str)
        selection = input('Enter number of the selection: ')
        try: 
            selection = int(selection) # Try converting the input to integer.
            if selection == 0:
                # create playlist with all songs in order
                return 'all', files
            elif selection == 1:
                # create playlist with all songs shuffled
                return 'shuffle', files
            elif selection == 2:
                # glue all songs together and play it
                return 'glued', files
            elif (selection>2 and selection<len(music_folders)+3): 
                # Single song selected.
                return 'single', files[selection-3]            
            else: 
                # Inputed a number not in the list of options.
                print("Error: Selection not in the valid list.") 
        except ValueError: 
            print("That's not a number") # If user didn't entered an integer

def choose_artist(parent_folder_id):

    files = GD_SVC.list_files_on_folder(parent_folder_id)
    artist_folders = [n for n in files if len(n['name'].split('.'))==1]
    
    while True:
        choice_str = """Choose a folder from the below list:\n"""
        for i, n in enumerate(artist_folders):
            choice_str += "{0}) {1}\n".format(i, n['name'])
        print(choice_str)
        selection = input('Enter number of the selected folder: ')
        try: 
            selection = int(selection) # Try converting the input to integer.
            if (selection>=0 and selection<len(artist_folders)): 
                # Sucessfull choice.
                print(" ")
                return artist_folders[selection]             
            else: 
                # Inputed a number not in the list of options.
                print("Error: Selection not in the valid list.") 
        except ValueError: 
            print("That's not a number") # If user didn't entered an integer

def choose_album(parent_folder_id):

    files = GD_SVC.list_files_on_folder(parent_folder_id)
    songs = sorted([f['name'] for f in files if f['name'].split('.')[-1] in SONG_FORMATS])

    if len(songs)>0:
        # There are songs inside this folder
        return None
    else:
        album_folders = [n for n in files if len(n['name'].split('.'))==1]
        while True:
            choice_str = """Choose a folder from the below list:\n"""
            for i, n in enumerate(album_folders):
                choice_str += "{0}) {1}\n".format(i, n['name'])
            print(choice_str)
            selection = input('Enter number of the selected music folder: ')
            try: 
                selection = int(selection) # Try converting the input to integer.
                if (selection>=0 and selection<len(album_folders)): 
                    # Sucessfull choice.
                    print(" ")
                    return album_folders[selection]             
                else: 
                    # Inputed a number not in the list of options.
                    print("Error: Selection not in the valid list.") 
            except ValueError: 
                print("That's not a number") # If user didn't entered an integer
                
def to_pydub_audiosegment(song_names, drive_songs_dict):
    print('Creating playlist data, it might take some time ... ')
    files = {}
    for n in song_names:
        for f in drive_songs_dict:
            if f['name'] == n:

                song_name = n.split('.')[0]
                file_fmt = n.split('.')[-1]

                file_data = GD_SVC.open_file(file_id=f['id'])
                song = AudioSegment.from_file(
                    io.BytesIO(file_data), 
                    format=file_fmt
                )
                files[song_name] = song
    return files
    
def launch_vlc_with_selection(selection_type, songs_inside_drive_folder):
    if selection_type in ['all', 'glued', 'single']:
        song_names = sorted(
            [
                f['name'] for f in songs_inside_drive_folder if f['name'].split('.')[-1] in SONG_FORMATS
            ]
        )
        song_info = to_pydub_audiosegment(song_names, songs_inside_drive_folder)
        play_files_as_playlist(song_info)
    else:
        song_names = random.shuffle(
            [
                f['name'] for f in songs_inside_drive_folder if f['name'].split('.')[-1] in SONG_FORMATS
            ]
        )
        song_info = to_pydub_audiosegment(song_names, songs_inside_drive_folder)
        play_files_continuously(song_info)

In [ ]:
artist = choose_artist(MUSIC_FOLDER)
album = choose_album(artist['id'])
selection_type, song_selection = choose_song(
    parent_folder_id=album['id'] if album else artist['id']
)
launch_vlc_with_selection(selection_type, song_selection)

Choose a folder from the below list:
0) Led Zeppelin
1) Opeth
2) Pink Floyd - The Wall [MFSL 1991] (1979) FLAC
3) Iron Maiden - Seventh Son of a Seventh Son

Enter number of the selected folder: 0
 
Choose a folder from the below list:
0) Led Zeppelin - Led Zeppelin IV (1971) [FLAC] {Deluxe Edition, 24-96, Highresaudio}
1) Led Zeppelin - Led Zeppelin II 1969 HIGHRESAUDIO 96
2) Led Zeppelin - Led Zeppelin 1969 HIGHRESAUDIO 96

Enter number of the selected music folder: 0
 

Choose a song from the below list:
0) Playlist with all songs in order
1) Playlist with all songs shuffled
2) All album glued as one continuous song
3) 01 - Black Dog.m4a
4) 02 - Rock And Roll.m4a
5) 03 - The Battle Of Evermore.m4a
6) 04 - Stairway To Heaven.m4a
7) 05 - Misty Mountain Hop.m4a
8) 06 - Four Sticks.m4a
9) 07 - Going To California.m4a
10) 08 - When The Levee Breaks.m4a

Enter number of the selection: 0
Creating playlist data, it might take some time ... 
exporting playlist to temp file ...
Playing on VLC

In [53]:
chosen_folder = album['id'] if album else artist['id']
files = GD_SVC.list_files_on_folder(chosen_folder)
data_files = [f for f in files if f['name'].split('.')[-1]=='pickle']
if len(data_files)>0:
    pickle_files = []
    for f in data_files:
        bytes_pkl = GD_SVC.open_as_bytes(f['id'])
        with open("escrom.pkl","rb") as f:
            file_pkl = pickle.load(bytes_pkl)
        #file_pkl = pickle.load(bytes_pkl)
        pickle_files.append(file_pkl)

FileNotFoundError: [Errno 2] No such file or directory: 'escrom.pkl'

In [23]:
artist

{'id': '1Oe1VwMzzLRDrQwpl40ezD8zK4VFr5_oS', 'name': 'Led Zeppelin'}

In [3]:
music_folders = GD_SVC.list_files_on_folder(MUSIC_FOLDER)

In [4]:
CHOSEN_ALBUM = music_folders[0]

In [5]:
CHOSEN_ALBUM

{'id': '1fuROMQV6D4-RyG50pClyaxN4Q6Rkxtyg', 'name': 'icono_blanco.png'}

In [6]:
songs = GD_SVC.list_files_on_folder(CHOSEN_ALBUM['id'])
names = sorted([f['name'] for f in songs if f['name'].split('.')[-1] in SONG_FORMATS])

In [7]:
songs

[]

In [9]:
os.getcwd()

'/Users/data/myStreamigSvc'

In [11]:
from googleapiclient.http import MediaFileUpload

In [13]:
media = MediaFileUpload(
    os.path.join(os.getcwd(),'icono_blanco.png'),
    mimetype='image/png'
)

In [14]:
file_metadata = {
    'name': 'icono_blanco.png',
    'parents': [MUSIC_FOLDER, CHOSEN_ALBUM]
}

In [15]:
file_metadata

{'name': 'icono_blanco.png',
 'parents': ['1XS5yDT8Ym3ME0SUywRuMFm5AGxaSIzaN',
  {'id': '1Oe1VwMzzLRDrQwpl40ezD8zK4VFr5_oS', 'name': 'Led Zeppelin'}]}

In [ ]:
self.drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()

In [19]:
GD_SVC.upload_file(
    full_path_to_file=os.path.join(os.getcwd(),'icono_blanco.png'),
    drive_file_name='icono_blanco.png', 
    mimetype=None,
    parent_folders_ids = [CHOSEN_ALBUM['id']]
)

image/png


'1zyN09kncpMB4wkZfVfuP9ZerWWRf_O2z'

In [8]:
names

[]

In [8]:
song_data = to_pydub_audiosegment(song_names=names, drive_songs_dict=songs)

No data about current album. Creating data ...


In [9]:
song_data

{'01 - The Leper Affinity': <pydub.audio_segment.AudioSegment at 0x110fccc90>}

In [10]:
from tempfile import TemporaryDirectory
import pickle 
import os

In [13]:
with TemporaryDirectory() as td:
    with open(os.path.join(td,'songs_data.pickle'), 'wb') as handle:
        pickle.dump(song_data, handle)

    GD_SVC.upload_file(
        full_path_to_file=os.path.join(td,'songs_data.pickle'),
        drive_file_name='songs_data.pickle', 
        mimetype=None,
        parent_folders_ids = [MUSIC_FOLDER, CHOSEN_ALBUM]
    )

In [11]:
song_info = get_songs_audio_segments(song_names=names, files_inside_drive=songs)

In [12]:
song_info

In [59]:
album_chosen = choose_album_folder(parent_folder_id=music_folder)
song_chosen = choose_song(album_chosen['id'])

Choose a folder from the below list:
0) Opeth
1) Pink Floyd - The Wall [MFSL 1991] (1979) FLAC
2) Iron Maiden - Seventh Son of a Seventh Son

Enter number of the selected music folder: 0
 

Choose a song from the below list:
0) Playlist with all songs
1) All album glued as one continuous song
2) 01 - The Leper Affinity.flac
3) 01 - The Leper Affinity.m4a

Enter number of the selection: 2
 


In [60]:
song_chosen

{'id': '1Hd4xLYPw4m3fv73dwjWI_tIXrqqfhD4Q', 'name': 'Opeth'}

In [25]:
files_inside = drive_svc.files().list(
    q = "'{folderId}' in parents".format(folderId=choice['id']),
    #q="mimeType='image/jpeg'",
    spaces='drive',
    fields='nextPageToken, files(id, name)',
    pageToken=None
).execute()

In [26]:
#song_formats = ['flac', 'm4a', 'mp3', 'wav']
song_formats = ['m4a']
names = sorted([f['name'] for f in files_inside['files'] if f['name'].split('.')[-1] in song_formats])

In [27]:
names

['01 - The Leper Affinity.m4a']

In [28]:
files = {}
for n in names:
    for f in files_inside['files']:
        if f['name'] == n:
            
            song_name = n.split('.')[0]
            file_fmt = n.split('.')[-1]
            
            file_data = gdrive_open_file(file_id=f['id'])
            song = AudioSegment.from_file(
                io.BytesIO(file_data), 
                format=file_fmt
            )
            files[song_name] = song

In [29]:
files.keys()

dict_keys(['01 - The Leper Affinity'])

In [33]:
play_files_as_playlist(files)

exporting playlist to temp file ...
Playing on VLC ...


KeyboardInterrupt: 